In [8]:
# !pip install keras
# !pip install tensorflow
# !pip install h5py

    100% |################################| 5.3MB 287kB/s eta 0:00:01


In [1]:
import keras
from keras.applications.resnet50 import ResNet50

import numpy as np

Using TensorFlow backend.


In [4]:
import numpy as np
import torch

def pyt_to_keras(pytorch_model, keras_model):
  """
    Given a PyTorch model, this method transfers the weight to
    a Keras Model (with backend TensorFlow) with the same architecture.
    Assumptions:
      1. The corresponding layer names in both the models will be the same
      2. Will throw KeyError when layer is there in Keras model but not in PyTorch model,
        otherwise, will ignore the layer.
    Implementated Layers:
      1. 2D Convolutional Layer
      2. Fully Connected Layer
    Eg:
    class PyNet(nn.Module):
      def __init__(self):
          super(PyNet, self).__init__()
          self.conv_1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(3, 4), padding=0)
      def forward(self, x):
          return self.conv_1(nn.ZeroPad2d((1, 2, 1, 1))(x))
    pyt_model = PyNet()
    a = keras.Input(shape=(5, 6, 1), name='input')
    b = keras.layers.Conv2D(2, (3, 4), activation='linear', padding='same',
                            name='conv_1', bias_initializer='random_uniform')(a)
    keras_model = keras.models.Model(inputs=a, outputs=b)
    keras_model = pyt_to_keras(pyt_model, keras_model)
    Args:
    :param pytorch_model: Similar model in PyTorch
    :type pytorch_model: torch.nn.Module
    :param keras_model: Model loaded in Keras.
    :type keras_model: keras.models.Model
    :return: PyTorch model with weight transferred from the Keras Model
    :rtype: torch.nn.Module
    """
  pyt_state_dict = pytorch_model.state_dict()
  for idx, layer in enumerate(keras_model.layers):
    if type(layer).__name__.endswith('Conv2D'):
      # Keras 2D Convolutional layer: height * width * input channels * output channels
      # PyTorch 2D Convolutional layer: output channels * input channels * height * width
      name = layer.name
      weights = np.transpose(pyt_state_dict[name + '.weight'].numpy(), (2, 3, 1, 0))
      bias = pyt_state_dict[name + '.bias'].numpy()
      keras_model.layers[idx].set_weights([weights, bias])
    elif type(layer).__name__.endswith('Dense'):
      # Keras Linear Layer: input neurons * output neurons
      # PyTorch Linear Layer: output neurons * input neurons
      name = layer.name
      weights = np.transpose(pyt_state_dict[name + '.weight'].numpy(), (1, 0))
      bias = pyt_state_dict[name + '.bias'].numpy()
      keras_model.layers[idx].set_weights([weights, bias])
  return keras_model

In [2]:
keras_model = ResNet50(weights='imagenet')

In [10]:
import torchvision.models as models
pytorch_model = models.resnet50(pretrained = True)

In [11]:
import torch.nn as nn

print(pytorch_model.fc)
pytorch_model.fc = nn.Linear(pytorch_model.fc.in_features, 10)
print(pytorch_model.fc)

Linear (2048 -> 1000)
Linear (2048 -> 10)


In [12]:
pytorch_model.load_state_dict(torch.load('/data/datasets/sound_datasets/pytorch_UrbanSound8K/saved_models/cvr_final_project/resnet50_v2_melspect_15_968.pt'))

In [15]:
# keras_pytorch_model = pyt_to_keras(pytorch_model,keras_model)

# Version 2 attempt

In [ ]:
from converter import pytorch_to_keras
k_model = pytorch_to_keras((10, 32, 32,), output) 

In [18]:
!python3 ./pytorch2keras/setup.py

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/pip/download.py", line 421, in get_file_content
    with open(url, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'requirements.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "./pytorch2keras/setup.py", line 9, in <module>
    reqs = [str(ir.req) for ir in install_reqs]
  File "./pytorch2keras/setup.py", line 9, in <listcomp>
    reqs = [str(ir.req) for ir in install_reqs]
  File "/usr/local/lib/python3.5/dist-packages/pip/req/req_file.py", line 84, in parse_requirements
    filename, comes_from=comes_from, session=session
  File "/usr/local/lib/python3.5/dist-packages/pip/download.py", line 425, in get_file_content
    'Could not open requirements file: %s' % str(exc)
pip.exceptions.InstallationError: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
